# Random Search

Ao invés de explorarmos exaustivamente nosso espaço de parâmetros, escolhemos combinações aleatoriamente e as testamos no nosso estimador. Dessa forma, não precisamos explorar o espaço inteiro para encontrar combinações de hiperparâmetros boas o suficiente para o problema em que queremos resolver. Não encontraremos necessáriamente um minimo/máximo global, mas pelo menos um local suficiente.

Diferente do GridSeatch que precisa de pontos específicos no espaço de parâmetros (1, 10, 25 ... n), o RandomSearch permite um espaço contínuo (0.5, 1, 2.558, 14 ... n).

In [ ]:
import pandas as pd
import numpy as np

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

dados_azar = dados.sort_values('vendido', ascending = True)
x_azar = dados_azar[['preco', 'idade_do_modelo', 'km_por_ano']]
y_azar = dados_azar['vendido']

def resultados_validacao_cruzada(busca, x, y):
    scores = cross_val_score(busca, x_azar, y_azar, cv = KFold(n_splits = 5, shuffle = True))

    intervalo_acuracia = [scores.mean()-(2*scores.std()), scores.mean()+(2*scores.std())]

    print("Melhor combinação: ", busca.best_params_)
    print("Resultados parciais: ", scores)
    print("Média de acurácia: %.2f%%" % (scores.mean() * 100))
    print("Intervalo de acurácia: %.2f%% ~ %.2f%%" % (intervalo_acuracia[0]*100, intervalo_acuracia[1]*100))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

SEED = 301
np.random.seed(SEED)

## espaço de parametros com 36 combinações possíveis:
## 2 x 3 x 3 x 2 = 36
espaco_de_parametros = {
    'max_depth' : [3, 5],
    'min_samples_split' : [32, 64, 128],
    'min_samples_leaf' : [32, 64, 128],
    'criterion' : ['gini', 'entropy']
}

busca = RandomizedSearchCV(DecisionTreeClassifier(),
                           espaco_de_parametros,
                           cv = KFold(n_splits = 5, shuffle = True),
                           random_state = SEED,
                           n_iter = 16) ## vamos rodar para apenas 16 combinações do espaço de 36
busca.fit(x_azar, y_azar)

resultados = pd.DataFrame(busca.cv_results_)

Nested Cross Validation do modelo com cross_val_score:

In [ ]:
resultados_validacao_cruzada(busca, x_azar, y_azar)

Melhor combinação:  {'min_samples_split': 64, 'min_samples_leaf': 32, 'max_depth': 3, 'criterion': 'gini'}
Resultados parciais:  [0.797  0.778  0.7915 0.774  0.7945]
Média de acurácia: 78.70%
Intervalo de acurácia: 76.85% ~ 80.55%


Com a busca aleatória de apenas 16 combinações, obtivemos uma acurácia muito parecida com o Grid Search, na qual procura a combinação no espaço inteiro de parâmetros.

# Customizando o espaço de parâmetros

In [ ]:
from scipy.stats import randint

# 7 x 96 x 96 x 2 = +129 mil combinações
espaco_de_parametros = {
    'max_depth' : [3, 5,10, 15, 20, 30, None],
    'min_samples_split' : randint(32, 128), # toda vez que rodar devolve um numero aleatorio entre 32 e 128
    'min_samples_leaf' : randint(32, 128),  # 128-32 = 96 números
    'criterion' : ['entropy', 'gini']
}

In [ ]:
busca = RandomizedSearchCV(DecisionTreeClassifier(),
                           espaco_de_parametros,
                           cv = KFold(n_splits = 5, shuffle = True),
                           random_state = SEED,
                           n_iter = 16) ## vamos rodar para apenas 16 combinações do espaço de 129 mil
busca.fit(x_azar, y_azar)

resultados = pd.DataFrame(busca.cv_results_)

In [ ]:
resultados_validacao_cruzada(busca, x_azar, y_azar)

Melhor combinação:  {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 71, 'min_samples_split': 100}
Resultados parciais:  [0.784  0.773  0.78   0.7895 0.789 ]
Média de acurácia: 78.31%
Intervalo de acurácia: 77.08% ~ 79.54%


Dessa forma, conseguimos extrair um bom resultado de um grid com mais de 129 mil combinações possíveis em muito menos tempo.

**Resumão das combinações ordenadas da melhor acurácia para a pior com desvio padrão e os parâmetros geradores:**

In [ ]:
resultados_ordenados_pela_media = resultados.sort_values("mean_test_score", ascending=False)
for indice, linha in resultados_ordenados_pela_media.iterrows():
    print("%.3f%% +-(%.3f%%) %s" % (linha.mean_test_score*100, linha.std_test_score*2, linha.params))

78.700% +-(0.004%) {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 71, 'min_samples_split': 100}
78.410% +-(0.010%) {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 73, 'min_samples_split': 72}
78.410% +-(0.010%) {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 64, 'min_samples_split': 67}
78.270% +-(0.011%) {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 125, 'min_samples_split': 59}
78.100% +-(0.014%) {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 124, 'min_samples_split': 88}
78.010% +-(0.004%) {'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 74, 'min_samples_split': 58}
77.930% +-(0.011%) {'criterion': 'entropy', 'max_depth': 15, 'min_samples_leaf': 126, 'min_samples_split': 84}
77.860% +-(0.011%) {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 108, 'min_samples_split': 110}
77.680% +-(0.011%) {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 100, 'min_samples_split': 84}
77.660% +-(0.01

Ou seja, quando comparado com o GridSearch, o RandomSearch tem uma vantagem bem grande que é o tempo de execução. Aumentando um pouco o espaço de parâmetros a gente aumenta considerávelmente o tempo de execução para o GridSearch, deixando essa execução muitas vezes inviável. Dessa forma, o RandomSearch consegue encontrar um resultado muito bom e parecido com muito menos tempo de processamento.